In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import gzip as gz
import ast
import re
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
info=[]
for i in gz.open(r'C:\Users\Usuario\Desktop\HENRY\gonzalo1\user_reviews.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))

df_user_reviews = pd.DataFrame(info)

df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
# Con las funciones explode y normalize desanidamos las columna "reviews" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_reviews_exploded = df_user_reviews.explode(['reviews'])
df_reviews_desanidado = pd.json_normalize(df_reviews_exploded['reviews'].dropna())

# Por otra parte reindexamos los DF
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_exploded.reset_index(inplace=True)

# Concatenamos y eliminamosla columna que originalmente desanidamos

user_reviews = pd.concat([df_reviews_exploded,df_reviews_desanidado], axis=1)
user_reviews = user_reviews.drop(columns = ['reviews'])
user_reviews

,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user_reviews_duplicados =  user_reviews.duplicated( keep="first")
user_reviews_duplicados.sum()

10

In [5]:
user_reviews = user_reviews.drop_duplicates(keep = "first")

In [6]:
user_reviews.dropna(inplace=True)

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
user_reviews['posted']

0         Posted November 5, 2011.
1            Posted July 15, 2011.
2           Posted April 21, 2011.
3            Posted June 24, 2014.
4        Posted September 8, 2013.
                   ...            
59300              Posted July 10.
59301               Posted July 8.
59302               Posted July 3.
59303              Posted July 20.
59304               Posted July 2.
Name: posted, Length: 59305, dtype: object

In [8]:
#Primer paso es extraer del la columna posted la fecha que esta en tipo de dato string, entonces puedo transformarla
user_reviews['posted'] = user_reviews['posted'].str.extract(r'Posted ([\w\s\d,]+)')
user_reviews.head(3)

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"November 5, 2011",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"July 15, 2011",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"April 21, 2011",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [9]:
#Reemplazo la palabra Posted por espacio vacio
user_reviews['posted'] = user_reviews['posted'].replace({'Posted': ''}, regex=True)
user_reviews.head(3)

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"November 5, 2011",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"July 15, 2011",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"April 21, 2011",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [10]:
#Transformo ahora la columna a tipo de dato datetime
user_reviews['posted'] = pd.to_datetime(user_reviews['posted'], errors='coerce')
user_reviews.head(3)

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [11]:
user_reviews = user_reviews.copy()

In [14]:
archivocsv = 'df/user_reviewsFinal.csv'
user_reviews.to_csv(archivocsv, index=False, encoding='utf-8')

In [15]:
#Transformo el archivo csv a parquet
#Leo el archivo csv
user_reviews= pd.read_csv("df/user_reviewsFinal.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "df/user_reviews.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(user_reviews)
pq.write_table(table,output_file)